# Gradio Demo: stream_asr

In [ ]:
!pip install -q gradio torch torchaudio transformers

In [ ]:
import gradio as gr
from transformers import pipeline
import numpy as np

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import pipeline, Conversation

chatbot = pipeline(model="facebook/blenderbot-400M-distill", device=0)
# Conversation objects initialized with a string will treat it as a user message
conversation = chatbot("I'm looking for a movie - what's your favourite one?")
conversation



[{'generated_text': " I don't really have a favorite movie, but I do like action movies. What about you?"}]

In [ ]:
def transcribe(stream, new_chunk):
    sr, y = new_chunk
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y
    speech_text = transcriber({"sampling_rate": sr, "raw": stream})["text"]
    response = chatbot(speech_text)
    return (
        stream,
        speech_text,
        response[-1]["generated_text"]
    )

In [ ]:


demo = gr.Interface(
    transcribe,
    inputs=["state", gr.Audio(sources=["microphone"], streaming=True)],
    outputs=["state", gr.Textbox(label="Recognized audio"), gr.Textbox(label="Bot's answer")],
    live=True,
)

if __name__ == "__main__":
    demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://22720c6b8478f06373.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://22720c6b8478f06373.gradio.live
